<a href="https://colab.research.google.com/github/stephenkati/Kiswahili-Mozilla-Common-Voice/blob/main/Kiswahili_NLP_ASR%2C_Translation_%26_Zero_Shot_Text_Classification_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://radiodownloaddw-a.akamaihd.net/downloads/Events/dwelle/dira/mp3/kis/E6F773D5_2.mp3
!ffmpeg -i  E6F773D5_2.mp3 -ar 16000 -ac 1 -f wav football.wav

--2023-10-21 07:35:32--  https://radiodownloaddw-a.akamaihd.net/downloads/Events/dwelle/dira/mp3/kis/E6F773D5_2.mp3
Resolving radiodownloaddw-a.akamaihd.net (radiodownloaddw-a.akamaihd.net)... 23.200.156.215, 23.200.156.210
Connecting to radiodownloaddw-a.akamaihd.net (radiodownloaddw-a.akamaihd.net)|23.200.156.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2951943 (2.8M) [audio/mpeg]
Saving to: ‘E6F773D5_2.mp3’

E6F773D5_2.mp3      100%[===================>]   2.81M  1.96MB/s    in 1.4s    

2023-10-21 07:35:35 (1.96 MB/s) - ‘E6F773D5_2.mp3’ saved [2951943/2951943]

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-lib

In [2]:
!pip install transformers
!pip install SoundFile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.4 MB/s eta 0:00:00


In [ ]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse

def parse_transcription(wav_file):
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained("alokmatta/wav2vec2-large-xlsr-53-sw")
    model = Wav2Vec2ForCTC.from_pretrained("alokmatta/wav2vec2-large-xlsr-53-sw")

    # load audio
    audio_input, sample_rate = sf.read(wav_file)

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

parse_transcription("football.wav")